In [1]:
import pandas as pd
import numpy as np

In [2]:
salida_paso1_preprocesar_s3 = "salida_paso1_preprocesar_s3_generar_periodods_invalidez/"
salida_paso2_preprocesar_s6_pandas = "salida_paso2_preprocesar_s6_pandas/"

s3_inhab = pd.read_pickle(salida_paso1_preprocesar_s3 + "inhabilitaciones.pkl")
s6_df = pd.read_hdf(salida_paso2_preprocesar_s6_pandas + "s6_hdf_dates.h5")

FileNotFoundError: [Errno 2] No such file or directory: '/salida_paso1_preprocesar_s3_generar_periodods_invalidez/inhabilitaciones.pkl'

In [21]:
s3_inhab = s3_inhab.dropna(subset=["sancion_nombre", "inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal"])

s3_inhab['inhabilitacion_fechaInicial'] = s3_inhab['inhabilitacion_fechaInicial'].apply(lambda x: pd.to_datetime(x, utc=True))
s3_inhab['inhabilitacion_fechaFinal'] = s3_inhab['inhabilitacion_fechaFinal'].apply(lambda x: pd.to_datetime(x, utc=True))

s3_inhab['inhabilitacion_fechaInicial'] = s3_inhab['inhabilitacion_fechaInicial'].dt.date
s3_inhab['inhabilitacion_fechaFinal'] = s3_inhab['inhabilitacion_fechaFinal'].dt.date

In [22]:
s6_df["parties_name"] = s6_df["parties_name"].str.lower().str.strip()
s6_df["parties_name"] = s6_df["parties_name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

s6_df["parties_contactPoint_name"] = s6_df["parties_contactPoint_name"].str.lower().str.strip()
s6_df["parties_contactPoint_name"] = s6_df["parties_contactPoint_name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [23]:
s3_inhab.head()

,id,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona
7,29,SE/18/2019/JQII/AR,alejandro valerio payan,F,-,2020-10-23,2024-10-22,particular
8,28,SE/18/2019/JQII/AR,"horus grupo empresarial, sociedad anonima de c...",M,LA CONSTRUCCION Y DISEÑO DE TODA CLASE DE OBRA...,2020-10-23,2025-12-07,particular
10,655d1ba40e5b49001df642b2,01/2019-SU2-PR,francisco isaac oliva marquez,F,N/A,2023-05-16,2027-05-16,particular
11,006410016/2021,006410016/2021,"areo calafia, s.a. de c.v.",Dato no proporcionado,Dato no proporcionado,2022-08-16,2023-02-16,particular
12,000270050/2021,000270050/2021,"office expres, s. de r.l. de c.v.",Dato no proporcionado,Dato no proporcionado,2022-08-25,2024-02-25,particular


In [24]:
s6_df.head()

,_id.$oid,ocid,id,parties_name,parties_contactPoint_name,earliest_contractPeriod_startDate,latest_contractPeriod_endDate
0,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,sesea aguascalientes,monica lizbeth de la cruz ortega,2022-06-14,2022-06-14
1,62a913e7db2a665c4fce6223,ocds-ty10ed-03-06-2022-10:13:43,11,nissan torres corzo,grupo torres corzo automotriz de aguascaliente...,2022-06-14,2022-06-14
2,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,sesea aguascalientes,monica lizbeth de la cruz ortega,2022-06-14,2022-06-14
3,62a913e7db2a665c4fce6226,ocds-ty10ed-07-06-2022-10:37:52,14,grant limpieza y mantenimiento,grant limpieza y mantenimiento,2022-06-14,2022-06-14
4,62a913e7db2a665c4fce6225,ocds-ty10ed-06-06-2022-08:06:10,13,sesea aguascalientes,monica lizbeth de la cruz ortega,2022-06-14,2022-06-14


In [25]:
df1 = s3_inhab.merge(s6_df, left_on="sancion_nombre", right_on="parties_name", how = "inner", suffixes = ("s3", "s6") )
df2 = s3_inhab.merge(s6_df, left_on="sancion_nombre", right_on="parties_contactPoint_name", how = "inner", suffixes = ("s3", "s6") )
df = pd.concat([df1, df2])
df = df.drop_duplicates()

In [26]:
df.shape

(3529, 15)

In [27]:
df = df.dropna(subset=["inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal", "earliest_contractPeriod_startDate", "latest_contractPeriod_endDate"])
df.shape

(3317, 15)

In [28]:
df.head(3)

,ids3,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona,_id.$oid,ocid,ids6,parties_name,parties_contactPoint_name,earliest_contractPeriod_startDate,latest_contractPeriod_endDate
0,091200002/2022,091200002/2022,"cia. constructora y supervisora heos, s.a. de ...",Dato no proporcionado,Dato no proporcionado,2022-08-16,2022-11-16,particular,607093b808d6775517794f0f,ocds-07smqs-1581454,SHCP-1581454-2021-04-05,"cia. constructora y supervisora heos, s.a. de ...",None,2018-03-16,2019-03-30
1,091200002/2022,091200002/2022,"cia. constructora y supervisora heos, s.a. de ...",Dato no proporcionado,Dato no proporcionado,2022-08-16,2022-11-16,particular,607093b808d6775517794f15,ocds-07smqs-1581468,SHCP-1581468-2021-04-05,"cia. constructora y supervisora heos, s.a. de ...",None,2018-02-23,2019-03-09
6,091200002/2022,091200002/2022,"cia. constructora y supervisora heos, s.a. de ...",Dato no proporcionado,Dato no proporcionado,2022-08-16,2022-11-16,particular,6070943f437a064a6054eca6,ocds-07smqs-1872249,SHCP-1872249-2021-04-05,"cia. constructora y supervisora heos, s.a. de ...",None,2019-05-10,2020-02-15


In [29]:
mask1 = df.earliest_contractPeriod_startDate.between(df.inhabilitacion_fechaInicial, df.inhabilitacion_fechaFinal)
mask2 = df.latest_contractPeriod_endDate.between(df.inhabilitacion_fechaInicial, df.inhabilitacion_fechaFinal)
final_mask = mask1 | mask2
df["contrato_durante_inhabilitacion"] = final_mask
df["contrato_durante_inhabilitacion"] = df["contrato_durante_inhabilitacion"].astype(int)
df_result = df[final_mask]
#df_result = df

In [30]:
df_result[["inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal", "earliest_contractPeriod_startDate", "latest_contractPeriod_endDate"]]

,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,earliest_contractPeriod_startDate,latest_contractPeriod_endDate
150,2022-08-16,2022-11-16,2022-07-01,2022-09-28
151,2022-08-16,2022-11-16,2022-07-01,2022-09-28
211,2021-06-25,2022-06-25,2021-10-20,2021-12-22
212,2021-06-25,2022-06-25,2021-10-19,2021-12-31
213,2021-06-25,2022-06-25,2021-10-18,2021-12-22
...,...,...,...,...
357,2004-09-07,2024-08-07,2018-04-30,2018-12-31
358,2014-12-01,2024-08-01,2018-02-01,2018-07-15
359,2014-12-01,2024-08-01,2018-07-01,2018-12-31
360,2014-12-01,2024-08-01,2018-02-01,2018-07-15


In [31]:
df_result.head()

,ids3,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona,_id.$oid,ocid,ids6,parties_name,parties_contactPoint_name,earliest_contractPeriod_startDate,latest_contractPeriod_endDate,contrato_durante_inhabilitacion
150,091200008/2021,091200008/2021,"ecotono estudios ambientales, s.a. de c.v.",Dato no proporcionado,"LLEVAR A CABO LA PLANEACI�N, DIRECCI�N, SUPERV...",2022-08-16,2022-11-16,particular,6453e7a55ea335c07065bd62,ocds-07smqs-2462531,SHCP-2462531-2022-09-07,"ecotono estudios ambientales, s.a. de c.v.",None,2022-07-01,2022-09-28,1
151,091200008/2021,091200008/2021,"ecotono estudios ambientales, s.a. de c.v.",Dato no proporcionado,"LLEVAR A CABO LA PLANEACI�N, DIRECCI�N, SUPERV...",2022-08-16,2022-11-16,particular,6453ed9e5ea335c07073a307,-2462531,-2462531-2023-01-09,"ecotono estudios ambientales, s.a. de c.v.",None,2022-07-01,2022-09-28,1
211,184740001/2021,184740001/2021,industria delta manufacturing sa de cv,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,particular,6453e6bc5ea335c07063b0b6,ocds-07smqs-2342381,SHCP-2342381-2022-09-07,industria delta manufacturing sa de cv,None,2021-10-20,2021-12-22,1
212,184740001/2021,184740001/2021,industria delta manufacturing sa de cv,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,particular,6453e6ae5ea335c070639254,ocds-07smqs-2335889,SHCP-2335889-2022-09-07,industria delta manufacturing sa de cv,None,2021-10-19,2021-12-31,1
213,184740001/2021,184740001/2021,industria delta manufacturing sa de cv,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,particular,6453e6b45ea335c070639fde,ocds-07smqs-2338937,SHCP-2338937-2022-09-07,industria delta manufacturing sa de cv,None,2021-10-18,2021-12-22,1


In [32]:
df_result.columns

Index(['ids3', 'expediente', 'sancion_nombre', 'sancion_tipoPersona',
       'sancion_objetoSocial', 'inhabilitacion_fechaInicial',
       'inhabilitacion_fechaFinal', 'tipo_persona', '_id.$oid', 'ocid', 'ids6',
       'parties_name', 'parties_contactPoint_name',
       'earliest_contractPeriod_startDate', 'latest_contractPeriod_endDate',
       'contrato_durante_inhabilitacion'],
      dtype='object')

In [33]:
columnas_orden = ["sancion_nombre",'tipo_persona', 'sancion_tipoPersona', 'sancion_objetoSocial', 'inhabilitacion_fechaInicial',
                  'inhabilitacion_fechaFinal','earliest_contractPeriod_startDate', 'latest_contractPeriod_endDate', 'parties_name', "parties_contactPoint_name",
                  'ids3', 'expediente', '_id.$oid', 'ocid', 'ids6', 'contrato_durante_inhabilitacion']
df_result = df_result[columnas_orden]
df_result = df_result.reset_index(drop = True)

In [34]:
df_result.head()

,sancion_nombre,tipo_persona,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,earliest_contractPeriod_startDate,latest_contractPeriod_endDate,parties_name,parties_contactPoint_name,ids3,expediente,_id.$oid,ocid,ids6,contrato_durante_inhabilitacion
0,"ecotono estudios ambientales, s.a. de c.v.",particular,Dato no proporcionado,"LLEVAR A CABO LA PLANEACI�N, DIRECCI�N, SUPERV...",2022-08-16,2022-11-16,2022-07-01,2022-09-28,"ecotono estudios ambientales, s.a. de c.v.",None,091200008/2021,091200008/2021,6453e7a55ea335c07065bd62,ocds-07smqs-2462531,SHCP-2462531-2022-09-07,1
1,"ecotono estudios ambientales, s.a. de c.v.",particular,Dato no proporcionado,"LLEVAR A CABO LA PLANEACI�N, DIRECCI�N, SUPERV...",2022-08-16,2022-11-16,2022-07-01,2022-09-28,"ecotono estudios ambientales, s.a. de c.v.",None,091200008/2021,091200008/2021,6453ed9e5ea335c07073a307,-2462531,-2462531-2023-01-09,1
2,industria delta manufacturing sa de cv,particular,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,2021-10-20,2021-12-22,industria delta manufacturing sa de cv,None,184740001/2021,184740001/2021,6453e6bc5ea335c07063b0b6,ocds-07smqs-2342381,SHCP-2342381-2022-09-07,1
3,industria delta manufacturing sa de cv,particular,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,2021-10-19,2021-12-31,industria delta manufacturing sa de cv,None,184740001/2021,184740001/2021,6453e6ae5ea335c070639254,ocds-07smqs-2335889,SHCP-2335889-2022-09-07,1
4,industria delta manufacturing sa de cv,particular,Dato no proporcionado,"FABRICACION DE PIEZAS, REFACCIONES, MAQUINARIA...",2021-06-25,2022-06-25,2021-10-18,2021-12-22,industria delta manufacturing sa de cv,None,184740001/2021,184740001/2021,6453e6b45ea335c070639fde,ocds-07smqs-2338937,SHCP-2338937-2022-09-07,1


In [35]:
df_result.to_excel("contrato_durante_inhabilitacion_IntegriDAT.xlsx", index=False)